# Wasserstein Loss and Gradient Penalty

In the course, we discussed the limitations of the binary cross entropy loss (BCE Loss). It can lead to:
* Mode collapse: when the generator is stuck in a single mode of the distribution it is trying to replicate. For example, a generator training on the MNIST dataset would be stuck in generating images of certain digits only, as shown below.

<img src='assets/collapse.png' width=50% />

* Vanishing gradient: it's a common problem with many neural network architectures, but is very common when training GANs. Because the discriminator's task is much easier than the generator's, the discriminator tends to converge faster and reach a high accuracy. The discriminator loss gets close to zero and the gradients become very small, leading to that vanishing gradient problem. 

In this notebook, you will:
* implement the Wasserstein Loss
* implement two types of gradient penalties



## Wasserstein Loss

The Wasserstein GAN paper introduced a new type of loss function: the Wasserstein Distance. 

We are now reshaping the problem GANs are solving: instead of having a loss function that classifies a distribution as being real or not, we have a loss function that tries to minimize the distance between the real and the fake distribution. The difference is subtle but plays a big role in the stability of GANs.

<img src='assets/gradient_replace.png' width=90% />

The discriminator is now called a **critic** because its job is not really to distinguish between real and fake anymore but to maximize the distance between the two distributions. However, we will be using both terms interchangeably for the sake of clarity. 

The Wasserstein loss can be calculated using the formula below:

<center>$\min_{g} \max_{c} E(c(x)) - E(c(g(z)))$</center>

You are now familiar with the minimax function. The main difference with the BCE Loss is the disapperance of the logs!

### First exercise: Implement the Wasserstein Loss

The Wasserstein Loss (W-Loss) is taking the vector of logits outputed by the discriminator as input. In comparison, the BCE Loss was taking the probabilities (logits after a softmax layer) as inputs. The discriminator W-Loss is trying to maximize the mean value of the logits of real images and minize the mean value of the logits of fake images. The generator W-Loss is trying to maximize the mean value of the logits of fake images.


In [ ]:
import torch

import tests

In [ ]:
def disc_w_loss(real_logits: torch.Tensor, fake_logits: torch.Tensor):
    """
    Wasserstein Discriminator Loss
    
    args:
    - real_logits: vector of logits outputed by the discriminator with a real input image
    - fake_logits: vector of logits outputed by the discriminator with a fake input image 
    """
    ####
    # IMPLEMENT HERE
    ####
    return

In [ ]:
def disc_g_loss(fake_logits: torch.Tensor):
    """
    Wasserstein Generator Loss
    
    args:
    - fake_logits: vector of logits outputed by the discriminator with a fake input image 
    """
    ####
    # IMPLEMENT HERE
    ####
    return

In [ ]:
tests.check_disc_w_loss(disc_w_loss)

In [ ]:
tests.check_gen_w_loss(disc_g_loss)

## Gradient penalty

To train a GAN with the Wasserstein Loss, the discriminator (or critic) must be 1-Lipschitz continuous. 

The 1-Lipschitz continuity constraint implies that the norm of the gradient of the function must be below 1. In other words, for a function $f(x)$:

</br>
<center>$|| \frac{df}{dx} || < 1$</center>

Because the W-Loss is not bounded between 0 and 1 like the BCE loss, the above constraint makes sure that the loss does not grow too much. 

In the original paper, the authors enforced this condition by using weight clipping.

## WGAN-GP
Introducing Wasserstein Gan with Gradient Penalty, or WGAN-GP. In this paper, the authors introduce a more robust way to enforce the 1-Lipschitz constaint of the critic: a **gradient penalty term** in the loss function. 

The gradient penalty is calculated as follows:
* sample a random point $\hat{x}$ between the generated distribution and the real distribution. 
* run this sample through the discriminator and calculate the gradient $\nabla_{\hat{x}} D(\hat{x})$
* calculate the L2 norm of the gradient $|| \nabla_{\hat{x}} D(\hat{x}) ||_{2}$
* remove 1, square the result and calculate the mean $(|| \nabla_{\hat{x}} D(\hat{x}) ||_{2} - 1) ^{2}$

### Second exercise [Optional]: Implement the gradient penalty

In the second exercise of this notebook, you will implement the above gradient penalty. To help you, I have created a dummy critic module.

<br>
<br>
<details>
<summary>
<font size="3" color="black">
<b>Click for tips</b>
</font>
</summary>

* To calculate the gradients, you first have to set the attribute of a tensor `requires_grad` to True.
* You can use the following code to calculate the gradients:
```
torch.autograd.grad(critic(x), x, grad_outputs=torch.ones_like(critic(x)), create_graph=True)[0]
```
</details>

In [ ]:
import torch.nn as nn

In [ ]:
class Critic(nn.Module):
    """ 
    Dummy critic class 
    """
    def __init__(self):
        super(Critic, self).__init__()
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return torch.pow(x, 2)

In [ ]:
def gradient_penalty(real_sample: torch.Tensor, 
                     fake_sample: torch.Tensor,
                     critic: nn.Module) -> torch.Tensor:
    """
    Gradient penalty of the WGAN-GP model
    args:
    - real_sample: sample from the real dataset
    - fake_sample: generated sample
    
    returns:
    - gradient penalty
    """
    ####
    # IMPLEMENT HERE
    ####
    return

In [ ]:
real_sample = torch.randn(3, 32, 32)
fake_sample = torch.randn(3, 32, 32)
critic = Critic()

gradient_penalty = gradient_penalty(real_sample, fake_sample, critic)

## DRAGAN

The DRAGAN paper offered a different approach to calculate the gradient penalty and enforce the 1-Lipschitz constraint on the critic.

(N. Kodali, J. Abernethy, et al, "*ON CONVERGENCE AND STABILITY OF GANS*", College of Computing Georgia Institute of Technology [Online]Available at: arxiv.org/pdf/1705.07215.pdf \[Accessed 06/30/2022\])

<img src='assets/dragan_gp.png' width=80% />

As you can see, the formula is very similar, especially since the authors use $k = 1$ for their experiments. The main difference with the WGAN-GP gradient penalty is the $\delta$ term, which is a noise term. In their implementation, the authors calculate $X_{p} = X + \delta $ as follows:

<center>
    $X_{p} = X + 0.5 * \sigma({X}) * N$ 
</center>

where $\sigma$ is the standard deviation and $N$ a noise term sampled from the uniform distribution.

The gradient penalty is then calculated as follows:
* sample a random point $\hat{x}$ between the real distribution $X$ and $X_{p}$ . 
* run this sample through the discriminator and calculate the gradient $\nabla_{\hat{x}} D(\hat{x})$
* calculate the L2 norm of the gradient $|| \nabla_{\hat{x}} D(\hat{x}) ||_{2}$
* remove 1, square the result and calculate the mean $(|| \nabla_{\hat{x}} D(\hat{x}) ||_{2} - 1) ^{2}$


### BCE Loss
Interestingly, using this gradient penalty lifts some of the constraint on the BCE Loss and the authors use the above gradient penalty with the vanilla GAN losses (BCE Loss).

### Third exercise  [Optional]: Implement the DRAGAN gradient penalty

In the third exercise of this notebook, you will implement the DRAGAN gradient penalty. This is a one liner difference with the above implementation!

In [ ]:
def gradient_penalty_dragan(real_sample: torch.Tensor, critic: nn.Module) -> torch.Tensor:
    """
    Gradient penalty of the WGAN-GP model
    args:
    - real_sample: sample from the real dataset
    
    returns:
    - gradient penalty
    """
    ####
    # IMPLEMENT HERE
    ####
    return

In [ ]:
dragan_gp = gradient_penalty_dragan(real_sample, critic)

# WARNING

The gradient penalty terms penalize each input to the critic individually. Therefore, the critic should map a single input to a single output. However, we use some layers in the discriminator that remove this property: the BatchNormalization layers. 

Keep this in mind if you decide to use the gradient penalty in your project! 